In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
from tqdm import tqdm
from keras import optimizers

from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import time


Using TensorFlow backend.


In [2]:
x_train = []
x_test = []
y_train = []

In [3]:
df_train = pd.read_csv('planet/train_classes.csv')
df_test = pd.read_csv('planet/sample_submission.csv')

In [4]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

In [5]:
labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [6]:
for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('planet/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (64, 64)))
    y_train.append(targets)

for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('planet/test-jpg/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (64, 64)))
    

100%|███████████████████████████████████████████████████████████████████████████| 61191/61191 [01:52<00:00, 541.53it/s]


In [7]:
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)/255.
x_test  = np.array(x_test, np.float32)/255.

print(x_train.shape)
print(y_train.shape)

(40479, 64, 64, 3)
(40479, 17)


In [8]:
nfolds = 5

num_fold = 0
sum_score = 0
#input_channels=3

yfull_test = []
yfull_train =[]

In [9]:
from keras.applications.vgg19 import VGG19
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [10]:
input_size = 64
input_channels = 3
batch_size = 64

In [11]:
kf = KFold(n_splits=nfolds, shuffle=True, random_state=1)


In [12]:
for train_index, test_index in kf.split(x_train):
        start_time_model_fitting = time.time()
        
        X_train = x_train[train_index]
        Y_train = y_train[train_index]
        X_valid = x_train[test_index]
        Y_valid = y_train[test_index]

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        kfold_weights_path = os.path.join('', 'weights_kfold_' + str(num_fold) + '.h5')
        
        base_model = VGG19(include_top=False,
                       weights='imagenet',
                       input_shape=(input_size,input_size,input_channels))
        
        model = Sequential()
        model.add(BatchNormalization(input_shape=(input_size, input_size,input_channels)))
        model.add(base_model)
        '''model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(256, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        '''
        model.add(Flatten())
        #model.add(Dense(512, activation='relu'))
        #model.add(BatchNormalization())
        #model.add(Dropout(0.5))
        model.add(Dense(17, activation='sigmoid'))

        epochs_arr = [50, 5, 5]
        learn_rates = [0.001, 0.0001, 0.00001]

        for learn_rate, epochs in zip(learn_rates, epochs_arr):
            opt  = optimizers.Adam(lr=learn_rate)
            model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                          optimizer=opt,
                          metrics=['accuracy'])
            callbacks = [EarlyStopping(monitor='val_loss', patience=4, verbose=0),
                         ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.1,
                                   patience=2,
                                   cooldown=2,
                                   verbose=1),
            ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)]

            model.fit(x = X_train, y= Y_train, validation_data=(X_valid, Y_valid),
                  batch_size= batch_size, verbose=2, epochs=epochs,callbacks=callbacks,shuffle=True)
        
        if os.path.isfile(kfold_weights_path):
            model.load_weights(kfold_weights_path)
        
        p_valid = model.predict(X_valid, batch_size = batch_size, verbose=4)
        print(fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=4, average='samples'))

        p_train = model.predict(x_train, batch_size =batch_size, verbose=4)
        yfull_train.append(p_train)
        
        p_test = model.predict(x_test, batch_size = batch_size, verbose=4)
        yfull_test.append(p_test)


Start KFold number 1 from 5
Split train:  32383 32383
Split valid:  8096 8096
Train on 32383 samples, validate on 8096 samples
Epoch 1/50
 - 501s - loss: 0.1884 - accuracy: 0.9256 - val_loss: 0.1618 - val_accuracy: 0.9320
Epoch 2/50
 - 1058s - loss: 0.1542 - accuracy: 0.9371 - val_loss: 0.1441 - val_accuracy: 0.9414
Epoch 3/50
 - 1058s - loss: 0.1421 - accuracy: 0.9419 - val_loss: 0.1366 - val_accuracy: 0.9445
Epoch 4/50
 - 1058s - loss: 0.1364 - accuracy: 0.9447 - val_loss: 0.1346 - val_accuracy: 0.9459
Epoch 5/50
 - 651s - loss: 0.1322 - accuracy: 0.9461 - val_loss: 0.1301 - val_accuracy: 0.9473
Epoch 6/50
 - 1058s - loss: 0.1279 - accuracy: 0.9480 - val_loss: 0.1253 - val_accuracy: 0.9497
Epoch 7/50
 - 1057s - loss: 0.1235 - accuracy: 0.9505 - val_loss: 0.1246 - val_accuracy: 0.9511
Epoch 8/50
 - 1058s - loss: 0.1206 - accuracy: 0.9521 - val_loss: 0.1233 - val_accuracy: 0.9516
Epoch 9/50
 - 1057s - loss: 0.1169 - accuracy: 0.9538 - val_loss: 0.1218 - val_accuracy: 0.9522
Epoch 10/50

Epoch 15/50
 - 1058s - loss: 0.0899 - accuracy: 0.9647 - val_loss: 0.1076 - val_accuracy: 0.9591

Epoch 00015: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 16/50
 - 1057s - loss: 0.0860 - accuracy: 0.9664 - val_loss: 0.1075 - val_accuracy: 0.9594
Epoch 17/50
 - 1058s - loss: 0.0852 - accuracy: 0.9668 - val_loss: 0.1078 - val_accuracy: 0.9594
Train on 32383 samples, validate on 8096 samples
Epoch 1/5
 - 1061s - loss: 0.0868 - accuracy: 0.9660 - val_loss: 0.1085 - val_accuracy: 0.9591
Epoch 2/5
 - 1060s - loss: 0.0844 - accuracy: 0.9670 - val_loss: 0.1095 - val_accuracy: 0.9591
Epoch 3/5
 - 1059s - loss: 0.0815 - accuracy: 0.9681 - val_loss: 0.1106 - val_accuracy: 0.9591

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 4/5
 - 1060s - loss: 0.0767 - accuracy: 0.9701 - val_loss: 0.1114 - val_accuracy: 0.9593
Epoch 5/5
 - 1059s - loss: 0.0759 - accuracy: 0.9704 - val_loss: 0.1123 - val_accuracy: 0.9591
Train on 32383 samples

ResourceExhaustedError:  OOM when allocating tensor with shape[64,64,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradients_13/vgg19_4/block1_conv2/convolution_grad/Conv2DBackpropInput (defined at C:\Users\HP\miniconda3\envs\nlpp\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_592906]

Function call stack:
keras_scratch_graph


In [ ]:
result = np.array(yfull_test[0])
for i in range(1, nfolds):
    result += np.array(yfull_test[i])
result /= nfolds
result = pd.DataFrame(result, columns = labels)
result


In [ ]:
from tqdm import tqdm
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

In [ ]:
df_test['tags'] = preds
df_test.to_csv('submission_keras55.csv', index=False)